In [1]:
"""
Baseline for CIL project on road segmentation.
This simple baseline consits of a CNN with two convolutional+pooling layers with a soft-max loss
"""

import gzip
import os
import sys
import urllib
import matplotlib.image as mpimg
from PIL import Image
from random import shuffle

import code
import time

import tensorflow.python.platform

import numpy as np
import tensorflow as tf

import cil_helper as cil

from skimage.feature import greycomatrix, greycoprops

In [2]:
try:
    tf.app.flags.DEFINE_string('log_dir', '/tmp/tensorflow_lukas_balz',
                           """Directory where to write event logs """
                           """and checkpoint.""")
except: 
    print(tf.app.flags.FLAGS.log_dir)
    
FLAGS = tf.app.flags.FLAGS

In [3]:
TRAINING_SIZE = 5
VALIDATION_SIZE = 10  # Size of the validation set.
TEST_SIZE = 50
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 64 # 64
NUM_EPOCHS = 50
RESTORE_MODEL = False # If True, restore existing model instead of training a new one
RECORDING_STEP = 100
ERROR_STEP = 320

# Set image patch size in pixels
# IMG_PATCH_SIZE should be a multiple of 4
# image size should be an integer multiple of this number!
IMG_PATCH_SIZE = 4

# all prediction size
ALL_PER = 100

In [4]:
data_dir = 'training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/' 
test_dir = "test/"

In [5]:
NUM_CHANNELS = 6 # RGB, Saturation, Lightness, 4 Textur

In [6]:
# Extract label images
def extract_labels(filename, num_images):
    """Extract the labels into a 1-hot matrix [image index, label index]."""
    gt_imgs = []
    for i in range(1, num_images+1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            #print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            gt_imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    num_images = len(gt_imgs)
    gt_patches = [cil.img_crop(gt_imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    data = np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])
    labels = np.asarray([value_to_class(np.mean(data[i])) for i in range(len(data))])

    # Convert to dense 1-hot representation.
    return labels.astype(np.float32)

In [7]:
def make_patches(imgs):
    num_images = len(imgs)
    IMG_WIDTH = imgs[0].shape[0]
    IMG_HEIGHT = imgs[0].shape[1]
    DIMS = 4
    N_PATCHES_PER_IMAGE = (IMG_WIDTH/IMG_PATCH_SIZE)*(IMG_HEIGHT/IMG_PATCH_SIZE)

    img_patches = [cil.img_crop(imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    data = [img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))]

    return np.asarray(data)

In [8]:
# Assign a label to a patch v
def value_to_class(v):
    foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch
    df = np.sum(v)
    if df < foreground_threshold:
       return [0, 1]
    else:
       return [1, 0]

In [9]:
def load_preproc(save_dir):
    assert save_dir.endswith('/')
    return np.load(save_dir + 'preprocessed2.npy')

In [10]:
train_preproc = load_preproc(train_data_filename)
train_data = make_patches(train_preproc)

train_labels = extract_labels(train_labels_filename, TRAINING_SIZE)

In [11]:
print(train_data.shape)
print(train_labels.shape)

(1000000, 4, 4, 6)
(50000, 2)


In [12]:
c0 = 0
c1 = 0
for i in range(len(train_labels)):
    if train_labels[i][0] == 1:
        c0 = c0 + 1 # foreground -> road
    else:
        c1 = c1 + 1 # background
print ('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

Number of data points per class: c0 = 11292 c1 = 38708


In [13]:
sampling = False

In [14]:
print ('Balancing training data...')
min_c = min(c0, c1)
max_c = max(c0, c1)
print(min_c)
print(max_c)
idx0 = [i for i, j in enumerate(train_labels) if j[0] == 1]
idx1 = [i for i, j in enumerate(train_labels) if j[1] == 1]

if sampling: #sample smaller class
    if c0 < c1:
        while len(idx0) < c1:
            print(len(idx0))
            idx0.extend(idx0)
            print(len(idx0))
    else:
        while len(idx1)<c0:
            idx1.extend(idx1)
    lim = max_c
else: 
    lim = min_c
    
new_indices = idx0[0:lim] + idx1[0:lim]
train_data = train_data[new_indices,:,:,:]
train_labels = train_labels[new_indices]
    
print(len(new_indices))
print(train_data.shape)

Balancing training data...
11292
38708
22584
(22584, 4, 4, 6)


In [15]:
train_size = train_labels.shape[0]

In [16]:
# This is where training samples and labels are fed to the graph.
# These placeholder nodes will be fed a batch of training data at each
# training step using the {feed_dict} argument to the Run() call below.
train_data_node = tf.placeholder(
    tf.float32,
    shape=(BATCH_SIZE, IMG_PATCH_SIZE, IMG_PATCH_SIZE, NUM_CHANNELS),
    name='train_data')
train_labels_node = tf.placeholder(
    tf.float32,
    shape=(BATCH_SIZE, cil.NUM_LABELS),
    name='train_labels')
train_all_data_node = tf.constant(train_data)

In [17]:
# The variables below hold all the trainable weights. They are passed an
# initial value which will be assigned when when we call:
# {tf.initialize_all_variables().run()}
conv1_weights = cil.weight_variable([3, 3, NUM_CHANNELS, 512])
conv1_biases = cil.bias_variable([512])

conv2_weights = cil.weight_variable([2, 2, 512, 256])
conv2_biases = cil.bias_variable([256])

conv3_weights = cil.weight_variable([2, 2, 256, 128])
conv3_biases = cil.bias_variable([128])

fc1_weights = tf.Variable(  # fully connected, depth 512.
    tf.truncated_normal([int(IMG_PATCH_SIZE / 4 * IMG_PATCH_SIZE / 4 * 128), 1024],
                        stddev=0.1,
                        seed=SEED))
fc1_biases = tf.Variable(tf.constant(0.1, shape=[1024]))
fc2_weights = tf.Variable(
    tf.truncated_normal([1024, cil.NUM_LABELS],
                        stddev=0.1,
                        seed=SEED))
fc2_biases = tf.Variable(tf.constant(0.1, shape=[cil.NUM_LABELS]))
conv1_weights_hist = tf.histogram_summary("conv1_weights", conv1_weights)
conv2_weights_hist = tf.histogram_summary("conv2_weights", conv2_weights)
conv1_biases_hist = tf.histogram_summary("conv1_biases", conv1_biases)
conv2_biases_hist = tf.histogram_summary("conv2_biases", conv2_biases)
fc1_weights_hist = tf.histogram_summary("fc1_weights", fc1_weights)
fc1_biases_hist = tf.histogram_summary("fc1_biases", fc1_biases)
fc2_weights_hist = tf.histogram_summary("fc2_weights", fc2_weights)
fc2_biases_hist = tf.histogram_summary("fc2_biases", fc2_biases)

In [18]:
# We will replicate the model structure for the training subgraph, as well
# as the evaluation subgraphs, while sharing the trainable parameters.
def model(data, train=False):
    """The Model definition."""
    conv = cil.conv2d(data, conv1_weights)
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv1_biases))
    pool = cil.max_pool_2x2(relu)

    conv2 = cil.conv2d(pool, conv2_weights)
    relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))
    #pool2 = cil.max_pool_2x2(relu2)
    
    conv3 = cil.conv2d(relu2, conv3_weights)
    relu3 = tf.nn.relu(tf.nn.bias_add(conv3, conv3_biases))
    pool3 = cil.max_pool_2x2(relu3)

    # Reshape the feature map cuboid into a 2D matrix to feed it to the
    # fully connected layers.
    pool_shape = pool3.get_shape().as_list()
    reshape = tf.reshape(
        pool3,
        [pool_shape[0], pool_shape[1] * pool_shape[2] * pool_shape[3]])
    # Fully connected layer. Note that the '+' operation automatically
    # broadcasts the biases.
    
    print(pool3.get_shape())
    print(reshape.get_shape())
    print(fc1_weights.get_shape())
    
    fc1 = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
    if train:
        fc1 = tf.nn.dropout(fc1, 0.3, seed=SEED)
    fc2 = tf.matmul(fc1, fc2_weights) + fc2_biases

    return fc2

In [19]:
# Training computation: logits + cross-entropy loss.
logits = model(train_data_node, True) # BATCH_SIZE*NUM_LABELS
# print 'logits = ' + str(logits.get_shape()) + ' train_labels_node = ' + str(train_labels_node.get_shape())
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits, train_labels_node))
#loss = tf.reduce_mean(tf.square(logits - train_labels_node))
loss_summary = tf.scalar_summary('loss', loss)

(64, 1, 1, 128)
(64, 128)
(128, 1024)


In [20]:
# L2 regularization for the fully connected parameters.
regularizers = (tf.nn.l2_loss(fc1_weights) + tf.nn.l2_loss(fc1_biases) +
                tf.nn.l2_loss(fc2_weights) + tf.nn.l2_loss(fc2_biases))
# Add the regularization term to the loss.
loss += 5e-4 * regularizers

In [21]:
# Optimizer: set up a variable that's incremented once per batch and
# controls the learning rate decay.
batch = tf.Variable(0)
# Decay once per epoch, using an exponential schedule starting at 0.01.
learning_rate = tf.train.exponential_decay(
    0.01,                # Base learning rate.
    batch * BATCH_SIZE,  # Current index into the dataset.
    train_size,          # Decay step.
    0.95,                # Decay rate.
    staircase=True)
learning_rate_summary = tf.scalar_summary('learning_rate', learning_rate)

In [22]:
# Use simple momentum for the optimization.
#optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(loss, global_step=batch)

# try adam
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=batch)

# ada delta
#optimizer = tf.train.AdadeltaOptimizer(learning_rate).minimize(loss)

In [23]:
# Predictions for the minibatch, validation set and test set.
train_prediction = tf.nn.softmax(logits)
# We'll compute them only once in a while by calling their {eval()} method.
#all_shape = train_all_data_node.get_shape()
#all_ind = int(int(all_shape[0])*ALL_PER)
train_all_prediction = tf.nn.softmax(model(train_all_data_node[0:ALL_PER,:,:,:]))
#train_all_prediction = tf.nn.softmax(model(train_all_data_node))

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

(100, 1, 1, 128)
(100, 128)
(128, 1024)


In [24]:
s = tf.InteractiveSession()

In [25]:
# Get prediction overlaid on the original image for given input image
def get_prediction_with_overlay(img):
    
    img_prediction = get_prediction(img)
    oimg = cil.make_img_overlay(img[:,:,:3], img_prediction)

    return oimg

# Get a concatenation of the prediction and groundtruth for given input file
def get_prediction_with_groundtruth(img):
    
    img_prediction = get_prediction(img)
    cimg = cil.concatenate_images(img[:,:,:3], img_prediction)

    return cimg

# Get prediction for given input image 
def get_prediction(img):
    data = np.asarray(cil.img_crop(img, IMG_PATCH_SIZE, IMG_PATCH_SIZE))
    data = data[:,:,:,0:NUM_CHANNELS]
    data_node = tf.constant(data)
    output = tf.nn.softmax(model(data_node))
    output_prediction = s.run(output)
    img_prediction = cil.label_to_img(img.shape[0], img.shape[1], IMG_PATCH_SIZE, IMG_PATCH_SIZE, output_prediction)

    return img_prediction

In [26]:
if RESTORE_MODEL:
    # Restore variables from disk.
    saver.restore(FLAGS.train_dir + "/model.ckpt")
    print("Model restored.")

In [27]:
# Run all the initializers to prepare the trainable parameters.
tf.initialize_all_variables().run()

# Build the summary operation based on the TF collection of Summaries.
#summary_op = tf.merge_summary([loss_summary, learning_rate_summary, all_params_summary, conv1_weights_hist, conv2_weights_hist])
#summary_writer = tf.train.SummaryWriter(FLAGS.log_dir,
                                                #graph=s.graph)
print ('Initialized!')

Initialized!


In [28]:
# Loop through training steps.
print ('Total number of iterations = ' + str(int(NUM_EPOCHS * train_size / BATCH_SIZE)))

training_indices = range(train_size)

for iepoch in range(NUM_EPOCHS):

    # Permute training indices
    perm_indices = np.random.permutation(training_indices)

    for step in range (int(train_size / BATCH_SIZE)):

        offset = (step * BATCH_SIZE) % (train_size - BATCH_SIZE)
        batch_indices = perm_indices[offset:(offset + BATCH_SIZE)]

        # Compute the offset of the current minibatch in the data.
        # Note that we could use better randomization across epochs.
        batch_data = train_data[batch_indices, :, :, :]
        batch_labels = train_labels[batch_indices]
        
        #print('batch_data.shape = ' + str(batch_data.shape))
        # This dictionary maps the batch data (as a numpy array) to the
        # node in the graph is should be fed to.
        feed_dict = {train_data_node: batch_data, train_labels_node: batch_labels}

        if step % ERROR_STEP == 0:
            start_time = time.time()
            l, lr, all_predictions = s.run(
                        [loss, learning_rate, train_all_prediction],
                        feed_dict=feed_dict)
            
            duration = time.time() - start_time
            #print ('Minibatch loss: %.3f, learning rate: %.6f' % (l, lr))
            #print ('Minibatch error: %.1f%%' % cil.error_rate(predictions,
                                                              #   batch_labels))
            format_str = ('step %d, error: %.1f%% (%.3f sec/batch)')
            stepNr = step + iepoch*int(train_size / BATCH_SIZE)
            loss_value = cil.error_rate(all_predictions, train_labels)
            
            print (format_str % (stepNr, loss_value, duration)) 
        elif step % RECORDING_STEP == 0:
            
            start_time = time.time()
            l, lr, all_predictions = s.run(
                        [loss, learning_rate, train_prediction],
                        feed_dict=feed_dict)
            
            duration = time.time() - start_time
            #print ('Minibatch loss: %.3f, learning rate: %.6f' % (l, lr))
            #print ('Minibatch error: %.1f%%' % cil.error_rate(predictions,
                                                              #   batch_labels))
            format_str = ('step %d, batch error: %.1f%% (%.3f sec/batch)')
            stepNr = step + iepoch*int(train_size / BATCH_SIZE)
            loss_value = cil.error_rate(predictions, batch_labels)
            
            print (format_str % (stepNr, loss_value, duration)) 
        else:
            # Run the graph and fetch some of the nodes.
            _, l, lr, predictions = s.run(
                        [optimizer, loss, learning_rate, train_prediction],
                        feed_dict=feed_dict)

Total number of iterations = 17643
step 0, error: 100.0% (0.060 sec/batch)
step 100, batch error: 42.2% (0.038 sec/batch)
step 200, batch error: 50.0% (0.020 sec/batch)
step 300, batch error: 45.3% (0.018 sec/batch)
step 320, error: 100.0% (0.032 sec/batch)
step 352, error: 100.0% (0.027 sec/batch)
step 452, batch error: 48.4% (0.019 sec/batch)
step 552, batch error: 46.9% (0.020 sec/batch)


KeyboardInterrupt: 

In [ ]:
save_path = saver.save(s, FLAGS.train_dir + "/model4.ckpt")

In [29]:
print ("Running prediction on training set")
prediction_training_dir = "predictions_training/"
if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)
    
for i in range(TRAINING_SIZE):
    pimg = get_prediction_with_groundtruth(train_preproc[i])
    Image.fromarray(pimg).save(prediction_training_dir + "prediction4_" + str(i+1) + ".png")
    oimg = get_prediction_with_overlay(train_preproc[i])
    oimg.save(prediction_training_dir + "overlay4_" + str(i+1) + ".png")  

Running prediction on training set
(10000, 1, 1, 128)
(10000, 128)
(128, 1024)
(10000, 1, 1, 128)
(10000, 128)
(128, 1024)
(10000, 1, 1, 128)
(10000, 128)
(128, 1024)
(10000, 1, 1, 128)
(10000, 128)
(128, 1024)
(10000, 1, 1, 128)
(10000, 128)
(128, 1024)
(10000, 1, 1, 128)
(10000, 128)
(128, 1024)
(10000, 1, 1, 128)
(10000, 128)
(128, 1024)
(10000, 1, 1, 128)
(10000, 128)
(128, 1024)
(10000, 1, 1, 128)
(10000, 128)
(128, 1024)
(10000, 1, 1, 128)
(10000, 128)
(128, 1024)


In [ ]:
print ("Running prediction on test set")
prediction_test_dir = "predictions_test/"
if not os.path.isdir(prediction_test_dir):
    os.mkdir(prediction_test_dir)

test_preproc = load_preproc(test_dir)
for i in range(TEST_SIZE):
    pred = get_prediction(test_preproc[i])
    pimg = cil.img_float_to_uint8(pred)
    Image.fromarray(pimg).save(prediction_test_dir + "prediction4_" + str(i+1) + ".png")
    oimg = cil.make_img_overlay(test_preproc[i], pred)
    oimg.save(prediction_test_dir + "overlay4_" + str(i+1) + ".png")

In [ ]:
import re
th = 0.25

In [ ]:
# assign a label to a patch
def patch_to_label(patch):
    df = np.mean(patch)
    if df > th:
        return 1
    else:
        return 0


def mask_to_submission_strings(image_filename):
    """Reads a single image and outputs the strings that should go into the submission file"""
    img_number = int(re.search(r"\d+", image_filename).group(0))
    im = mpimg.imread(image_filename)
    patch_size = 16
    for j in range(0, im.shape[1], patch_size):
        for i in range(0, im.shape[0], patch_size):
            patch = im[i:i + patch_size, j:j + patch_size]
            label = patch_to_label(patch)
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))


def masks_to_submission(submission_filename, *image_filenames):
    """Converts images into a submission file"""
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for fn in image_filenames[0:]:
            f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(fn))


def save(submission_filename):
    #subm#ission_filename = 'submission_07.csv'
    image_filenames = []
    for i in range(1, 51):
        imagename = 'prediction_' + str(i)
        image_filename = 'predictions_test/' + imagename + '.png'
        print(image_filename)
        image_filenames.append(image_filename)
    masks_to_submission(submission_filename, *image_filenames)
    
save('submission_lukas_4x4.csv')